In [0]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# LSTM for sequence classification in the IMDB dataset
import numpy
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence



In [45]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pickle

with open('/content/gdrive/My Drive/Colab Notebooks/reviews_amzn.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [47]:
print(data.head())
print(data.shape)

                                                text  y
0   witti littl book make son laugh loud recit ca...  1
1   rememb see show air televis year ago child si...  1
2   beetlejuic well written movi everyth excel ac...  1
3   twist rumplestiskin captur film star michael ...  1
4   beetlejuic excel funni movi keaton hilari wac...  1
(50000, 2)


In [0]:
data = data.sample(frac=1).reset_index(drop = True)

In [0]:
final_X = data['text']
final_y = data['y']

In [0]:
X_train = final_X[:40000]
X_test = final_X[40000:]
y_train = final_y[:40000]
y_test = final_y[40000:]

In [0]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)

In [0]:
X_test = tokenizer.texts_to_sequences(X_test)

In [63]:
print(X_train[45])
print(type(X_train[45]))
print(len(X_train[45]))

[11, 1, 150, 14, 100, 442, 1403, 23, 130, 17, 331, 244, 46, 522, 26, 220, 6, 351]
<class 'list'>
18


In [64]:
# truncate and/or pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[45])

(40000, 500)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [65]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(5000, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [66]:
model.fit(X_train, y_train, nb_epoch=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/5
40000/40000 [==============================] - 912s 23ms/step - loss: 0.3622 - acc: 0.8395
Epoch 2/5
40000/40000 [==============================] - 907s 23ms/step - loss: 0.2543 - acc: 0.9000
Epoch 3/5
40000/40000 [==============================] - 910s 23ms/step - loss: 0.2238 - acc: 0.9141
Epoch 4/5
40000/40000 [==============================] - 904s 23ms/step - loss: 0.1980 - acc: 0.9247
Epoch 5/5
40000/40000 [==============================] - 887s 22ms/step - loss: 0.1800 - acc: 0.9329
Accuracy: 87.82%


<h2> Model 2 (Two Lstm layers ) </h2>

In [71]:
embedding_vecor_length = 32
model1 = Sequential()
model1.add(Embedding(5000, embedding_vecor_length, input_length=max_review_length))
model1.add(LSTM(100, return_sequences=True))
model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_11 (LSTM)               (None, 500, 100)          53200     
_________________________________________________________________
lstm_12 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 293,701
Trainable params: 293,701
Non-trainable params: 0
_________________________________________________________________
None


In [72]:
model1.fit(X_train, y_train, nb_epoch=3, batch_size=64)
# Final evaluation of the model
scores = model1.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/3
40000/40000 [==============================] - 1795s 45ms/step - loss: 0.3593 - acc: 0.8402
Epoch 2/3
40000/40000 [==============================] - 1843s 46ms/step - loss: 0.2526 - acc: 0.9000
Epoch 3/3
40000/40000 [==============================] - 1841s 46ms/step - loss: 0.2231 - acc: 0.9139
Accuracy: 88.23%


<h1>Summary</h1>

1.   Model1 with one LSTM layer ran for 5 epochs go1 accuracy of 87.81
2.   Model2 with 2 LSTM layers ran for 3 epochs got accuracy of 88.23
3.   We can get high accuracies if we run for more epochs. 

